In [169]:
import numpy as np
import timeit

In [170]:
m = 2.5
k = 8

In [171]:
def generate_matrix_and_vector(n, dtype=np.float64):
    matrix = np.zeros((n,n), dtype=dtype)

    for i in range(1, n+1):
        for j in range(1, n+1):
            if i == j:
                matrix[i-1, j-1] = k
            elif j > i:
                matrix[i-1, j-1] = (-1) ** j * (m / j)
            elif j == i-1:
                matrix[i-1, j-1] = m/i


    vector = np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=dtype)

    return matrix, vector

In [172]:
def difference(v1, v2):
    differences = np.abs(v1 - v2)

    max_difference = np.max(differences)
    return max_difference

In [173]:
def jacobi_iteration_matrix(A):
    D = np.diag(np.diag(A))
    D_inv = np.linalg.inv(D)
    M = np.eye(A.shape[0]) - D_inv @ A
    return M

def spectral_radius(matrix):
    eigenvalues = np.linalg.eigvals(matrix)
    return max(abs(eigenvalues))

In [174]:
# for i in range(3, 200):
#     A, _ = generate_matrix_and_vector(i)
#     M = jacobi_iteration_matrix(A)
#     rho = spectral_radius(M)
#     print(f"n = {i}, rho = {rho}")

In [175]:
def jacobi_method(A, b, rho, typed=1, stop=1):
    iterations = 0
    if typed == 1:
        x = np.zeros_like(b)
    else:
        x = np.array([100 if i % 2 == 0 else -100 for i in range(b.shape[0])])
        
    D = np.diag(A)
    R = A - np.diagflat(D)
    while True:
        x_new = (b - R @ x) / D
        iterations += 1
        if stop == 1:
            if np.linalg.norm(x_new - x) < rho:
                break
        else:
            if np.linalg.norm(A @ x_new - b) < rho:
                break
        x = x_new
            
    return x_new, iterations

In [176]:
rhos = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9, 1e-11]

In [177]:
def generate_iterations_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]

    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            _, iterations = jacobi_method(A, b, rho=rho, typed=typed, stop=stop)
            row += f"& {iterations} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Liczba iteracji dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [178]:
def generate_difference_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]

    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            x, _ = jacobi_method(A, b, rho=rho, typed=typed, stop=stop)
            diff = difference(b, A @ x)
            row += f"& {diff:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Różnice dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [179]:
def generate_execution_time_table(typed, stop):
    table = "\\noindent\\begin{minipage}{\\linewidth}\n\\centering\n"
    table += "\\begin{tabular}{|c|" + "c|" * len(rhos) + "}\n\\hline\n"
    table += "\\multicolumn{1}{|c|}{} & \\multicolumn{" + str(len(rhos)) + "}{c|}{\\textbf{$\\rho$}} \\\\ \\cline{2-" + str(len(rhos) + 1) + "}\n"
    table += "\\multicolumn{1}{|c|}{\\multirow{-2}{*}{\\textbf{$n$}}} "
    for rho in rhos:
        table += f"& {rho} "
    table += "\\\\ \\hline\n"

    n_values = list(range(5, 431, 25)) + [500,1000]
    
    for n in n_values:
        row = f"{n} "
        for rho in rhos:
            A, b = generate_matrix_and_vector(n)
            exec_time = timeit.timeit(lambda: jacobi_method(A, b, rho=rho, typed=typed, stop=stop), number=1)
            row += f"& {exec_time:.8f} "
        row += "\\\\ \\hline\n"
        table += row

    table += "\\end{tabular}\n"
    table += "\\captionsetup{justification=centering}\n"
    if typed == 1 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [0, 0, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    elif typed == 1 and stop == 2:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [0, 0, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    elif typed == 2 and stop == 1:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [100, -100, ...] i pierwszego kryterium stopu}\n\\end{minipage}"
    else:
        table += "\\captionof{table}{Czasy wykonania w sekundach dla różnych wartości n dla wektora [100, -100, ...] i drugiego kryterium stopu}\n\\end{minipage}"
    return table

In [180]:
print(generate_iterations_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
30 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
55 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
80 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
130 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
155 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
180 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
230 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
255 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
280 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
330 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
355 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
380 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
405 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
430 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
500 & 3 & 

In [181]:
print(generate_difference_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.00586995 & 0.00013075 & 0.00000240 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.00697123 & 0.00014847 & 0.00000232 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.00804614 & 0.00015937 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.00945857 & 0.00017350 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.00997413 & 0.00017863 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.01041403 & 0.00018299 & 0.00000237 & 0.00000008 & 0.00000000 & 0.00000000 \\ \

In [182]:
print(generate_execution_time_table(1, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00011250 & 0.00011640 & 0.00006850 & 0.00007820 & 0.00009500 & 0.00010910 \\ \hline
30 & 0.00005170 & 0.00007180 & 0.00007200 & 0.00008330 & 0.00010820 & 0.00011490 \\ \hline
55 & 0.00007730 & 0.00006380 & 0.00013220 & 0.00017060 & 0.00017460 & 0.00015570 \\ \hline
80 & 0.00008080 & 0.00006700 & 0.00009820 & 0.00009830 & 0.00011380 & 0.00012440 \\ \hline
105 & 0.00038060 & 0.00071810 & 0.00089630 & 0.00105040 & 0.00110620 & 0.00137160 \\ \hline
130 & 0.00057150 & 0.00075180 & 0.00076270 & 0.00086530 & 0.00132250 & 0.00160300 \\ \hline
155 & 0.00038740 & 0.00067950 & 0.00079070 & 0.00072500 & 0.00093890 & 0.00123270 \\ \hline
180 & 0.00038100 & 0.00045280 & 0.00058510 & 0.00080420 & 0.00097770 & 0.00113510 \\ \

In [183]:
print(generate_iterations_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 2 & 4 & 7 & 9 & 12 & 14 \\ \hline
30 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
55 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
80 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
105 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
130 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
155 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
180 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
205 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
230 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
255 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
280 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
305 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
330 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
355 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
380 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
405 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
430 & 3 & 5 & 7 & 9 & 12 & 14 \\ \hline
500 & 3 & 

In [184]:
print(generate_difference_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.03203668 & 0.00078881 & 0.00000267 & 0.00000007 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.00586995 & 0.00013075 & 0.00000240 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.00697123 & 0.00014847 & 0.00000232 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.00804614 & 0.00015937 & 0.00000228 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.00883454 & 0.00016729 & 0.00000230 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.00945857 & 0.00017350 & 0.00000233 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.00997413 & 0.00017863 & 0.00000235 & 0.00000008 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.01041403 & 0.00018299 & 0.00000237 & 0.00000008 & 0.00000000 & 0.00000000 \\ \

In [185]:
print(generate_execution_time_table(1, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00011330 & 0.00007480 & 0.00010340 & 0.00016560 & 0.00015040 & 0.00017000 \\ \hline
30 & 0.00010410 & 0.00008950 & 0.00011030 & 0.00012940 & 0.00015190 & 0.00017050 \\ \hline
55 & 0.00009110 & 0.00007400 & 0.00009500 & 0.00014520 & 0.00016300 & 0.00017920 \\ \hline
80 & 0.00009210 & 0.00007680 & 0.00009360 & 0.00011010 & 0.00013430 & 0.00015640 \\ \hline
105 & 0.00057720 & 0.00128130 & 0.00143230 & 0.00175490 & 0.00231920 & 0.00307570 \\ \hline
130 & 0.00056210 & 0.00090000 & 0.00087830 & 0.00187430 & 0.00232650 & 0.00229080 \\ \hline
155 & 0.00059660 & 0.00082940 & 0.00109810 & 0.00127150 & 0.00151520 & 0.00225260 \\ \hline
180 & 0.00064980 & 0.00074810 & 0.00144500 & 0.00136970 & 0.00202160 & 0.00190110 \\ \

In [186]:
print(generate_iterations_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 5 & 8 & 10 & 13 & 15 & 18 \\ \hline
30 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
55 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
80 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
130 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
155 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
180 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
230 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
255 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
280 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
330 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
355 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
380 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
405 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
430 & 6 & 8 & 10

In [187]:
print(generate_difference_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.06864039 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.01973649 & 0.00032769 & 0.00000680 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.02118610 & 0.00032018 & 0.00000650 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.02204760 & 0.00031516 & 0.00000631 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.02314150 & 0.00030842 & 0.00000607 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.02353316 & 0.00030594 & 0.00000598 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.02386429 & 0.00030380 & 0.00000590 & 0.00000003 & 0.00000000 & 0.00000000 \\ \

In [188]:
print(generate_execution_time_table(2, 1))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00013540 & 0.00013450 & 0.00025660 & 0.00019050 & 0.00016710 & 0.00020370 \\ \hline
30 & 0.00011350 & 0.00012170 & 0.00013080 & 0.00020200 & 0.00016960 & 0.00014860 \\ \hline
55 & 0.00009170 & 0.00017800 & 0.00012030 & 0.00012260 & 0.00013070 & 0.00015200 \\ \hline
80 & 0.00009880 & 0.00011720 & 0.00013570 & 0.00013140 & 0.00013870 & 0.00015890 \\ \hline
105 & 0.00067070 & 0.00082350 & 0.00098810 & 0.00151860 & 0.00160540 & 0.00175840 \\ \hline
130 & 0.00074040 & 0.00081280 & 0.00091330 & 0.00137180 & 0.00135970 & 0.00220350 \\ \hline
155 & 0.00075700 & 0.00079060 & 0.00102800 & 0.00125930 & 0.00151410 & 0.00176410 \\ \hline
180 & 0.00068780 & 0.00074600 & 0.00108640 & 0.00125920 & 0.00141060 & 0.00170450 \\ \

In [189]:
print(generate_iterations_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
30 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
55 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
80 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
105 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
130 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
155 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
180 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
205 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
230 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
255 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
280 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
305 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
330 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
355 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
380 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
405 & 6 & 8 & 10 & 13 & 15 & 18 \\ \hline
430 & 6 & 8 & 10

In [190]:
print(generate_difference_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.01509448 & 0.00033275 & 0.00000748 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
30 & 0.01973649 & 0.00032769 & 0.00000680 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
55 & 0.02118610 & 0.00032018 & 0.00000650 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
80 & 0.02204760 & 0.00031516 & 0.00000631 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
105 & 0.02266334 & 0.00031142 & 0.00000617 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
130 & 0.02314150 & 0.00030842 & 0.00000607 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
155 & 0.02353316 & 0.00030594 & 0.00000598 & 0.00000003 & 0.00000000 & 0.00000000 \\ \hline
180 & 0.02386429 & 0.00030380 & 0.00000590 & 0.00000003 & 0.00000000 & 0.00000000 \\ \

In [191]:
print(generate_execution_time_table(2, 2))

\noindent\begin{minipage}{\linewidth}
\centering
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
\multicolumn{1}{|c|}{} & \multicolumn{6}{c|}{\textbf{$\rho$}} \\ \cline{2-7}
\multicolumn{1}{|c|}{\multirow{-2}{*}{\textbf{$n$}}} & 0.1 & 0.001 & 1e-05 & 1e-07 & 1e-09 & 1e-11 \\ \hline
5 & 0.00013430 & 0.00011100 & 0.00012670 & 0.00015290 & 0.00017620 & 0.00019910 \\ \hline
30 & 0.00009080 & 0.00009480 & 0.00018330 & 0.00014730 & 0.00018890 & 0.00021770 \\ \hline
55 & 0.00011170 & 0.00010340 & 0.00011650 & 0.00014220 & 0.00015910 & 0.00018550 \\ \hline
80 & 0.00011580 & 0.00010870 & 0.00012640 & 0.00020050 & 0.00017400 & 0.00019920 \\ \hline
105 & 0.00132680 & 0.00166560 & 0.00217130 & 0.00250230 & 0.00308810 & 0.00372290 \\ \hline
130 & 0.00115360 & 0.00138330 & 0.00150550 & 0.00218420 & 0.00249330 & 0.00296350 \\ \hline
155 & 0.00101780 & 0.00147950 & 0.00174210 & 0.00219270 & 0.00260990 & 0.00311770 \\ \hline
180 & 0.00106680 & 0.00135210 & 0.00156660 & 0.00222470 & 0.00235400 & 0.00294870 \\ \

In [192]:
def generate_latex_table_radius():
    n_values = list(range(5, 431, 25)) + [500,1000]
    table_rows = []

    for n in n_values:
        A, _ = generate_matrix_and_vector(n)
        M = jacobi_iteration_matrix(A)
        radius = spectral_radius(M)
        table_rows.append((n, radius))

    latex_code = "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{|c|c|}\n\\hline\nn & Promień spektralny \\\\ \\hline\n"

    for n, radius in table_rows:
        latex_code += f"{n} & {radius:.6f} \\\\ \\hline\n"

    latex_code += "\\end{tabular}\n\\caption{Promienie spektralne dla różnych wartości n}\n\\end{table}"

    return latex_code

In [193]:
print(generate_latex_table_radius())

\begin{table}[h!]
\centering
\begin{tabular}{|c|c|}
\hline
n & Promień spektralny \\ \hline
5 & 0.156250 \\ \hline
30 & 0.156250 \\ \hline
55 & 0.156250 \\ \hline
80 & 0.156250 \\ \hline
105 & 0.156250 \\ \hline
130 & 0.156250 \\ \hline
155 & 0.156250 \\ \hline
180 & 0.156250 \\ \hline
205 & 0.156250 \\ \hline
230 & 0.156250 \\ \hline
255 & 0.156250 \\ \hline
280 & 0.156250 \\ \hline
305 & 0.156250 \\ \hline
330 & 0.156250 \\ \hline
355 & 0.156250 \\ \hline
380 & 0.156250 \\ \hline
405 & 0.156250 \\ \hline
430 & 0.156250 \\ \hline
500 & 0.156250 \\ \hline
1000 & 0.156250 \\ \hline
\end{tabular}
\caption{Promienie spektralne dla różnych wartości n}
\end{table}
